In [22]:
import lightgbm as lgb

OSError: dlopen(/Users/tacofoil/Desktop/Education/Data Talks Club/MLOPs/project/weather-forecast/.venv/lib/python3.13/site-packages/lightgbm/lib/lib_lightgbm.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib
  Referenced from: <D44045CD-B874-3A27-9A61-F131D99AACE4> /Users/tacofoil/Desktop/Education/Data Talks Club/MLOPs/project/weather-forecast/.venv/lib/python3.13/site-packages/lightgbm/lib/lib_lightgbm.dylib
  Reason: tried: '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/local/lib/libomp/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/local/lib/libomp/libomp.dylib' (no such file), '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/local/lib/libomp/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/local/lib/libo

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


#import lightgbm as lgb
import mlflow
import mlflow.lightgbm
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score


XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/Users/tacofoil/Desktop/Education/Data Talks Club/MLOPs/project/weather-forecast/.venv/lib/python3.13/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib\n  Referenced from: <98D50080-9632-3EA4-B874-146E55453763> /Users/tacofoil/Desktop/Education/Data Talks Club/MLOPs/project/weather-forecast/.venv/lib/python3.13/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: tried: '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/homebrew/opt/libom

In [2]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("weather_forecasting")

2025/07/14 16:16:35 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/07/14 16:16:35 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

<Experiment: artifact_location=('/Users/tacofoil/Desktop/Education/Data Talks '
 'Club/MLOPs/project/weather-forecast/notebooks/mlruns/1'), creation_time=1752502595643, experiment_id='1', last_update_time=1752502595643, lifecycle_stage='active', name='weather_forecasting', tags={}>

In [ ]:
def read_dataframe(filename):
    df = pd.read_csv(filename)
    return df

In [4]:
df = pd.read_csv("/Users/tacofoil/Desktop/Education/Data Talks Club/MLOPs/project/weather-forecast/data/processed/final_weather_data.csv")
df

,datetime,hour,day,month,weekday,city,Latitude,Longitude,temperature,humidity,...,weather_description_thunderstorm with drizzle,weather_description_thunderstorm with heavy drizzle,weather_description_thunderstorm with heavy rain,weather_description_thunderstorm with light drizzle,weather_description_thunderstorm with light rain,weather_description_thunderstorm with rain,weather_description_tornado,weather_description_very heavy rain,weather_description_volcanic ash,weather_description_nan
0,2012-10-01 12:00:00,12,1,10,0,Albuquerque,35.084492,-106.651138,NaN,NaN,...,False,False,False,False,False,False,False,False,False,True
1,2012-10-01 12:00:00,12,1,10,0,Atlanta,33.749001,-84.387978,NaN,NaN,...,False,False,False,False,False,False,False,False,False,True
2,2012-10-01 12:00:00,12,1,10,0,Beersheba,31.251810,34.791302,NaN,NaN,...,False,False,False,False,False,False,False,False,False,True
3,2012-10-01 12:00:00,12,1,10,0,Boston,42.358429,-71.059769,NaN,NaN,...,False,False,False,False,False,False,False,False,False,True
4,2012-10-01 12:00:00,12,1,10,0,Charlotte,35.227089,-80.843132,NaN,NaN,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1629103,2017-11-30 00:00:00,0,30,11,3,San Francisco,37.774929,-122.419418,NaN,NaN,...,False,False,False,False,False,False,False,False,False,True
1629104,2017-11-30 00:00:00,0,30,11,3,Seattle,47.606209,-122.332069,283.04,75.0,...,False,False,False,False,False,False,False,False,False,False
1629105,2017-11-30 00:00:00,0,30,11,3,Tel Aviv District,32.083328,34.799999,NaN,NaN,...,False,False,False,False,False,False,False,False,False,True
1629106,2017-11-30 00:00:00,0,30,11,3,Toronto,43.700111,-79.416298,274.51,60.0,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1629108 entries, 0 to 1629107
Data columns (total 68 columns):
 #   Column                                                   Non-Null Count    Dtype  
---  ------                                                   --------------    -----  
 0   datetime                                                 1629108 non-null  object 
 1   hour                                                     1629108 non-null  int64  
 2   day                                                      1629108 non-null  int64  
 3   month                                                    1629108 non-null  int64  
 4   weekday                                                  1629108 non-null  int64  
 5   city                                                     1629108 non-null  object 
 6   Latitude                                                 1629108 non-null  float64
 7   Longitude                                                1629108 non-null  float64
 8   te

In [7]:
df.describe()


,hour,day,month,weekday,Latitude,Longitude,temperature,humidity,pressure,wind_speed,wind_direction
count,1.629108e+06,1.629108e+06,1.629108e+06,1.629108e+06,1.629108e+06,1.629108e+06,1.621078e+06,1.600457e+06,1.612428e+06,1.621115e+06,1.621133e+06
mean,1.150134e+01,1.572532e+01,6.648730e+00,2.997613e+00,3.706674e+01,-7.354467e+01,2.885958e+02,6.813581e+01,1.016521e+03,2.790038e+00,1.829256e+02
std,6.922323e+00,8.793825e+00,3.464869e+00,1.999911e+00,5.734176e+00,5.089048e+01,1.035149e+01,2.244515e+01,1.556569e+01,2.092585e+00,1.060446e+02
min,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,2.577427e+01,-1.231193e+02,2.423367e+02,5.000000e+00,8.000000e+02,0.000000e+00,0.000000e+00
25%,6.000000e+00,8.000000e+00,4.000000e+00,1.000000e+00,3.276613e+01,-1.054013e+02,2.818840e+02,5.300000e+01,1.012000e+03,1.000000e+00,9.400000e+01
50%,1.200000e+01,1.600000e+01,7.000000e+00,3.000000e+00,3.617043e+01,-8.647124e+01,2.895800e+02,7.200000e+01,1.017000e+03,2.000000e+00,1.900000e+02
75%,1.800000e+01,2.300000e+01,1.000000e+01,5.000000e+00,4.099821e+01,-7.487433e+01,2.962500e+02,8.700000e+01,1.024000e+03,4.000000e+00,2.700000e+02
max,2.300000e+01,3.100000e+01,1.200000e+01,6.000000e+00,4.924966e+01,3.521633e+01,3.212200e+02,1.000000e+02,1.100000e+03,5.000000e+01,3.600000e+02


In [ ]:
# Drop columns that are null in temperature
## Should be moved to a preprocessing step
df = df.dropna(subset=['temperature'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1621078 entries, 9 to 1629106
Data columns (total 68 columns):
 #   Column                                                   Non-Null Count    Dtype  
---  ------                                                   --------------    -----  
 0   datetime                                                 1621078 non-null  object 
 1   hour                                                     1621078 non-null  int64  
 2   day                                                      1621078 non-null  int64  
 3   month                                                    1621078 non-null  int64  
 4   weekday                                                  1621078 non-null  int64  
 5   city                                                     1621078 non-null  object 
 6   Latitude                                                 1621078 non-null  float64
 7   Longitude                                                1621078 non-null  float64
 8   tempera

In [11]:
# Feature Engineering Layer

TARGET = "temperature"

def feature_engineering(data: pd.DataFrame):
    d = data.copy()
    d = d.dropna(subset=[TARGET])                        # keep rows with a target
    # num_cols = d.select_dtypes("number").columns
    # d[num_cols] = d[num_cols].fillna(d[num_cols].median())   # median impute
    X = d.drop(columns=[TARGET, "datetime", "city"])
    y = d[TARGET]
    return X, y

X, y = feature_engineering(df)


In [13]:
X

,hour,day,month,weekday,Latitude,Longitude,humidity,pressure,wind_speed,wind_direction,...,weather_description_thunderstorm with drizzle,weather_description_thunderstorm with heavy drizzle,weather_description_thunderstorm with heavy rain,weather_description_thunderstorm with light drizzle,weather_description_thunderstorm with light rain,weather_description_thunderstorm with rain,weather_description_tornado,weather_description_very heavy rain,weather_description_volcanic ash,weather_description_nan
9,12,1,10,0,29.558050,34.948212,25.0,1011.0,8.0,360.0,...,False,False,False,False,False,False,False,False,False,False
36,13,1,10,0,35.084492,-106.651138,50.0,1024.0,4.0,360.0,...,False,False,False,False,False,False,False,False,False,False
37,13,1,10,0,33.749001,-84.387978,94.0,1006.0,3.0,110.0,...,False,False,False,False,False,False,False,False,False,False
38,13,1,10,0,31.251810,34.791302,50.0,984.0,1.0,135.0,...,False,False,False,False,False,False,False,False,False,False
39,13,1,10,0,42.358429,-71.059769,68.0,1014.0,3.0,60.0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1629100,0,30,11,3,38.627270,-90.197891,46.0,1021.0,3.0,100.0,...,False,False,False,False,False,False,False,False,False,False
1629101,0,30,11,3,29.424120,-98.493629,48.0,1021.0,1.0,180.0,...,False,False,False,False,False,False,False,False,False,False
1629102,0,30,11,3,32.715328,-117.157257,72.0,1017.0,2.0,210.0,...,False,False,False,False,False,False,False,False,False,False
1629104,0,30,11,3,47.606209,-122.332069,75.0,1028.0,4.0,100.0,...,False,False,False,False,False,False,False,False,False,False


In [14]:
y

9          309.10
36         285.12
37         294.03
38         307.59
39         287.17
            ...  
1629100    283.44
1629101    292.02
1629102    291.40
1629104    283.04
1629106    274.51
Name: temperature, Length: 1621078, dtype: float64

In [19]:
# Split the data into train, validation, and test sets
# 80% train_val, 20% test
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)

# From the 80% train_val, split 25% val = 20% of total (0.25 * 0.8)
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.25, random_state=42
)

# Sizes
print(f"Train: {len(X_train)} | Val: {len(X_val)} | Test: {len(X_test)}")
# Add percetage of total
print(f"Train: {len(X_train)/len(X)*100:.2f}% | Val: {len(X_val)/len(X)*100:.2f}% | Test: {len(X_test)/len(X)*100:.2f}%")


Train: 972646 | Val: 324216 | Test: 324216
Train: 60.00% | Val: 20.00% | Test: 20.00%


In [ ]:
mlflow.log_params({
    "train_size": len(X_train),
    "train_percentage": len(X_train)/len(X)*100,
    "val_size": len(X_val),
    "val_percentage": len(X_val)/len(X)*100,
    "test_size": len(X_test),
    "test_percentage": len(X_test)/len(X)*100
})


In [20]:
X_train.to_parquet("X_train.parquet")
X_val.to_parquet("X_val.parquet")
X_test.to_parquet("X_test.parquet")
y_train.to_frame().to_parquet("y_train.parquet")
y_val.to_frame().to_parquet("y_val.parquet")
y_test.to_frame().to_parquet("y_test.parquet")


In [ ]:
# Cell 7  — training + logging
mlflow.xgboost.autolog(  # note the flavor change
    log_input_examples=True,
    log_model_signatures=True,
)

with mlflow.start_run(run_name="xgb_temperature"):
    model = xgb.XGBRegressor(
        n_estimators=600,
        max_depth=8,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="reg:squarederror",
        tree_method="hist",          # fast on CPUs
        random_state=42,
    )
    model.fit(X_train, y_train)

    preds = model.predict(X_val)
    metrics = {
        "rmse": root_mean_squared_error(y_val, preds),
        "mae":  mean_absolute_error(y_val, preds),
        "r2":   r2_score(y_val, preds),
    }
    mlflow.log_metrics(metrics)
    mlflow.log_params({"train_size": len(X_tr), "val_size": len(X_val)})

    # Register the model (will create a version 1 the first time)
    mlflow.register_model(
        model_uri=f"runs:/{run.info.run_id}/model",
        name="weather_temperature_lgbm",
    )

metrics
